In [263]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. what for
1. 对revenue data的计算

# 1.

In [264]:
import sys
print(sys.path)
sys.path.append('D:\\Desktop\\PROJ\\PAProj\\data_handler')

['D:\\PyCharm\\plugins\\python\\helpers-pro\\jupyter_debug', 'D:\\PyCharm\\plugins\\python\\helpers\\pydev', 'D:\\Desktop\\PROJ\\PAProj\\data_handler', 'D:\\Desktop\\PROJ\\PAProj', 'D:\\Anaconda\\python39.zip', 'D:\\Anaconda\\DLLs', 'D:\\Anaconda\\lib', 'D:\\Anaconda', '', 'D:\\Anaconda\\Lib\\site-packages', 'D:\\Anaconda\\Lib\\site-packages\\win32', 'D:\\Anaconda\\Lib\\site-packages\\win32\\lib', 'D:\\Anaconda\\Lib\\site-packages\\Pythonwin', 'D:\\Anaconda\\Lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Administrator\\.ipython', 'D:\\Desktop\\PROJ\\PAProj\\data_handler']


In [567]:

seed = 407      # 3,31,204,223,407


noise_pct = 0.05  # 噪音占比:我们希望生成的data总体上最多浮动的百分比noise_pct
data_key_path = "../data/target_datakey_all.csv"
N_gaussians = 2  # nums of Gaussian kernels
SET_VAL = False
VOUCHER = True # Need or not the config about voucher

# Input path
target_path = r"../data/targets_all"
# GT-2 NN 产生的params
params_opitim_path_NN = r"../data/SA_PT/params_seed=" + str(seed) + ".csv"
# MDN 产生的params
params_MDN = r"../data/revenue/output_MDN_seed="+str(seed)+".csv"


# Output path
target_ER_path = "../data/revenue/target_ER.csv"
GT_1_ER_path = "../data/revenue/target_GT_1_ER.csv"
GT_2_ER_path = "../data/revenue/target_GT_2_ER_seed="+str(seed)+".csv"
MDN_ER_path = "../data/revenue/target_MDN_ER_seed="+str(seed)+".csv"

import numpy as np
import pandas as pd
import os
from metric_for_GTs_func import *
from tqdm import tqdm
import scipy.special
import torch
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

In [568]:
def setup_seed(seed):
    """
    Set seed
    Args:
        seed:
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(seed)
train_pct = 0.7
vali_pct = 0.2
test_pct = 0.1
seed

407

In [569]:
data_key = pd.read_csv(data_key_path,encoding="utf-8")
params_opitim_NN = pd.read_csv(params_opitim_path_NN,encoding="utf-8")
MDN_output = pd.read_csv(params_MDN,encoding="utf-8")


# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)
LEN_ALL = target_all_path.__len__()
# 使用全部的data
shuffled_indices = np.random.permutation(LEN_ALL)
if SET_VAL:
    train_idx = shuffled_indices[:int(train_pct * LEN_ALL)]
    tmp = int((train_pct + vali_pct) * LEN_ALL)
    val_idx = shuffled_indices[int(train_pct * LEN_ALL):tmp]
    test_idx = shuffled_indices[tmp:]
else:
    train_idx = shuffled_indices[:int(train_pct * LEN_ALL)]
    tmp = int((train_pct + vali_pct) * LEN_ALL)
    test_idx = shuffled_indices[int(train_pct * LEN_ALL):tmp]
    val_idx = shuffled_indices[tmp:]

## 1.2 筛选test set
1. 是否需要voucher类型的数据？

In [570]:
if VOUCHER:
    test_idx_len = len(test_idx)
else:
    vou = []
    vou_ls = data_key[data_key['desc'].str.contains('Voucher')].index.tolist()
    vou_ls_2 = data_key[data_key['desc'].str.contains('VOUCHER')].index.tolist()
    vou = vou_ls + vou_ls_2

    print("before:",len(test_idx))
    test_idx = [i for i in test_idx if i not in vou]
    test_idx_len = len(test_idx)
    print("after:",len(test_idx))


# 2. 计算revenue
1. 结果为3列：['revenue','retail','odds']
    - odds = revenue/retails
## 2.1 GT-1的ER
1. retail本身

In [523]:
def GT_1_ER(target,setting):

    b = np.array(setting.bidfee)               # bid fee (dollar)
    v = np.array(setting.retail)               # valuation
    d = np.array(setting.bidincrement)

    revenue = v
    return revenue, v, revenue/v

## 2.2 target revenue

In [524]:
def target_ER(target_df,setting):

    b = np.array(setting.bidfee)               # bid fee (dollar)
    v = np.array(setting.retail)               # valuation
    d = np.array(setting.bidincrement)

    # target_df_uniq = target_df.drop_duplicates(inplace=False,ignore_index=True)

    revenue = (b+d) * ((target_df.N*target_df.P).sum())

    return revenue, v, revenue / v


## 2.3 计算GT-1与 target ER
1. 这两个一起算是因为和seed无关

In [525]:
col_ER = ['test_idx','revenue','retail','odds']
col_config = ['desc','bidincrement','bidfee','retail']

revenue_target = np.array([0]*LEN_ALL*col_ER.__len__()).reshape(-1,len(col_ER))
revenue_target_pd = pd.DataFrame(revenue_target,columns=col_ER)

revenue_GT_1 = np.array([0]*LEN_ALL*col_ER.__len__()).reshape(-1,len(col_ER))
revenue_GT_1_pd = pd.DataFrame(revenue_target,columns=col_ER)


In [450]:
for i in tqdm(range(LEN_ALL)):
    target_path_i_path = os.path.join(target_path, target_all_path[i])
    target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
    config = data_key.iloc[i,:]

    # GT-1
    target = list(target_df.iloc[:,0])
    revenue_GT_1_pd.iloc[i,0] = i
    revenue_GT_1_pd.iloc[i,1:] = GT_1_ER(target, config)

    # target
    revenue_target_pd.iloc[i,0] = i
    revenue_target_pd.iloc[i,1:] = target_ER(target_df, config)


100%|██████████| 1276/1276 [00:01<00:00, 937.09it/s]


In [451]:
revenue_target_pd.head()

,test_idx,revenue,retail,odds
0,0,315.378261,499.99,0.630769
1,1,405.015000,169.99,2.382581
2,2,610.420408,299.99,2.034803
3,3,206.756341,89.99,2.297548
4,4,48.946154,59.99,0.815905


- save

In [452]:
# revenue_target_pd.to_csv(target_ER_path,encoding="utf-8",header=True,index=False)
# revenue_GT_1_pd.to_csv(GT_1_ER_path,encoding="utf-8",header=True,index=False)

## 2.4 计算GT-2的ER [seed 相关]

In [453]:
## 2.2 GT-2的ER
def get_ER_GT_2(T_target, U, LEN, TARGET=1):
    """

    Args:
        T_target:
        U:
        LEN:
        TARGET:
        eps:

    Returns:
    NLL metric: 正值
    """

    # Solve for P with length of LEN
    P = np.array([0.0] * (LEN + 1))
    P[0] = 0.0
    tmp = np.array([0.0] * (LEN + 3))  # tmp的大小不需要太精确
    tmp[0] = 1.0
    # 注意：P[i][t] = U[i][1]*U[i][2]*...*(1-U[i][t+1])
    for t in range(1, len(P)):
        tmp[t] = tmp[t - 1] * U[t]  # tmp[t]存了U从1到(t)的连乘积
        P[t] = (1 - U[t + 1]) * tmp[t]

    # DO NOT DELETE the P[0] here to keep with actual meaning,和实际意义保持一致
    # P = np.delete(P,[0],axis=0)

    revenue = sum(i * P[i] for i in range(len(P)))

    return revenue


def GT_2_ER(target, setting, alpha_i, labda_i):
    b = np.array(setting.bidfee)  # bid fee (dollar)
    v = np.array(setting.retail)  # valuation
    d = np.array(setting.bidincrement)

    LEN, T = get_LEN_T(v, b, d, max(target))

    # Solve for U
    U = get_U_GT2(LEN, v, d, b, alpha_i, labda_i, eps=0.)

    revenue = (b + d) * get_ER_GT_2(target, U, LEN, TARGET=1)

    return revenue, v, revenue / v

In [454]:
revenue_GT_2 = np.array([0]*test_idx_len*len(col_ER)).reshape(-1,len(col_ER))
revenue_GT_2_pd = pd.DataFrame(revenue_GT_2,columns=col_ER)

In [455]:
# For those in test set
for idx,i in enumerate(test_idx):
    target_path_i_path = os.path.join(target_path, target_all_path[i])
    target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

    target = list(target_df.iloc[:,0])
    config = data_key.iloc[i,:]

    # GT-2
    alpha_i = params_opitim_NN.loc[i,'alpha'].item()
    labda_i = params_opitim_NN.loc[i,'labda'].item()

    revenue_GT_2_pd.iloc[idx,0] = i
    revenue_GT_2_pd.iloc[idx,1:] = GT_2_ER(target, config, alpha_i, labda_i)


In [456]:
GT_2_ER_path

'../data/revenue/target_GT_2_ER_seed=204.csv'

In [325]:
revenue_GT_2_pd.head()

,test_idx,revenue,retail,odds
0,38,910.377864,599.00,1.519829
1,905,340.008964,299.99,1.133401
2,606,284.438960,219.99,1.292963
3,551,42.764691,29.99,1.425965
4,893,57.050809,49.99,1.141244


In [326]:
revenue_GT_2_pd.to_csv(GT_2_ER_path,encoding="utf-8",header=True,index=False)

## 2.5 计算MDN的ER [seed 相关]
1. where k > 0 is the shape parameter and λ > 0 is the scale parameter
2. mean= scale*gamma(1+1/shape)

In [571]:
revenue_MDN = np.array([0]*test_idx_len*col_ER.__len__()).reshape(-1,len(col_ER))
revenue_MDN_pd = pd.DataFrame(revenue_MDN,columns=col_ER)

In [572]:
def MDN_ER(mdn_output, setting):

    b = np.array(setting.bidfee)  # bid fee (dollar)
    v = np.array(setting.retail)  # valuation
    d = np.array(setting.bidincrement)

    # 1. 从MDN.pth得到结果的预测：即MDN的输出
    revenue = 0.
    for j in range(N_gaussians):

        pi = mdn_output[0+(j*3)]
        scale = mdn_output[1+(j*3)]
        shape = mdn_output[2+(j*3)]

        if pi > 0.1:
            revenue += pi*scale*scipy.special.gamma(1+1/shape)

    revenue = (b + d) * revenue

    return revenue, v, revenue / v

In [573]:
for idx,i in enumerate(test_idx):
    target_path_i_path = os.path.join(target_path, target_all_path[i])
    target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

    target = list(target_df.iloc[:,0])
    config = data_key.iloc[i,:]

    # MDN

    mdn_output = MDN_output.iloc[i,:]
    revenue_MDN_pd.iloc[idx,0] = i
    revenue_MDN_pd.iloc[idx,1:] = MDN_ER(mdn_output, config)


In [574]:
revenue_MDN_pd.head()

,test_idx,revenue,retail,odds
0,708,94.540570,37.50,2.521082
1,1127,58.395507,59.99,0.973421
2,469,67.009959,59.99,1.117019
3,905,486.595241,299.99,1.622038
4,732,319.465623,222.00,1.439034


In [575]:
MDN_ER_path

'../data/revenue/target_MDN_ER_seed=407.csv'

In [576]:
revenue_MDN_pd.to_csv(MDN_ER_path,encoding="utf-8",header=True,index=False)

# 3. 分析


In [577]:
revenue_target_pd = pd.read_csv(target_ER_path,encoding="utf-8")
revenue_GT_1_pd = pd.read_csv(GT_1_ER_path,encoding="utf-8")
revenue_GT_2_pd = pd.read_csv(GT_2_ER_path,encoding="utf-8")
revenue_MDN_pd = pd.read_csv(MDN_ER_path,encoding="utf-8")

- 对于GT1和target，取test set

In [578]:
revenue_target_pd.head()

,test_idx,revenue,retail,odds
0,0,283.343478,499.99,0.566698
1,1,378.405000,169.99,2.226043
2,2,599.436735,299.99,1.998189
3,3,99.801220,89.99,1.109026
4,4,33.680769,59.99,0.561440


In [579]:
# index = revenue_GT_1_pd.index.get_loc(test_idx.tolist())  #2是索引号

revenue_GT_1 = revenue_GT_1_pd[revenue_GT_1_pd['test_idx'].isin(test_idx)]
revenue_target = revenue_target_pd[revenue_target_pd['test_idx'].isin(test_idx)]
revenue_GT_1.reset_index(inplace=True,drop=True)
revenue_target.reset_index(inplace=True,drop=True)


## 3.2 test set RMSE and MSE

In [580]:
def get_rmse(target,data):
    target.reset_index(inplace=True,drop=True)
    data.reset_index(inplace=True,drop=True)

    if VOUCHER:
        rmse = sqrt(mean_squared_error(data.odds, target.odds))
        # rmse = sqrt(np.sum( (target.odds-data.odds)*(target.odds-data.odds))/target.shape[0] )
    else:
        # Not in vou
        data_wo_vou = data[~data['test_idx'].isin(vou)]
        target_wo_vou = target[~target['test_idx'].isin(vou)]

        rmse = sqrt(mean_squared_error(data_wo_vou.odds,target_wo_vou.odds))

    # rmse = mean_absolute_error(data.odds, target.odds)

    return rmse

def get_mse(target,data):
    target.reset_index(inplace=True,drop=True)
    data.reset_index(inplace=True,drop=True)

    if VOUCHER:
        mse = mean_squared_error(data.odds, target.odds)
    else:
        # Not in vou
        data_wo_vou = data[~data['test_idx'].isin(vou)]
        target_wo_vou = target[~target['test_idx'].isin(vou)]

        mse = mean_squared_error(data_wo_vou.odds,target_wo_vou.odds)

    return mse

In [581]:
col_rmse = ['GT-1','GT-2(NN)','MDN']
rmse_arr = np.array([0.]*1*3).reshape(-1,3)
mse_arr = np.array([0.]*1*3).reshape(-1,3)
rmse_pd = pd.DataFrame(rmse_arr,columns=col_rmse)
mse_pd = pd.DataFrame(mse_arr,columns=col_rmse)

In [582]:
revenue_MDN_pd.head()

,test_idx,revenue,retail,odds
0,708,94.540570,37.50,2.521082
1,1127,58.395507,59.99,0.973421
2,469,67.009959,59.99,1.117019
3,905,486.595241,299.99,1.622038
4,732,319.465623,222.00,1.439034


In [583]:
rmse_pd.iloc[0,0] = get_rmse(revenue_target, revenue_GT_1)
rmse_pd.iloc[0,1] = get_rmse(revenue_target, revenue_GT_2_pd)
rmse_pd.iloc[0,2] = get_rmse(revenue_target, revenue_MDN_pd)

In [584]:
mse_pd.iloc[0,0] = get_mse(revenue_target, revenue_GT_1)
mse_pd.iloc[0,1] = get_mse(revenue_target, revenue_GT_2_pd)
mse_pd.iloc[0,2] = get_mse(revenue_target, revenue_MDN_pd)

In [586]:
print(f"在testing上的RMSE,seed={seed}")
print(rmse_pd)
print(f"在testing上的MSE,seed={seed}")
print(mse_pd)

在testing上的RMSE,seed=407
       GT-1  GT-2(NN)       MDN
0  0.859185  3.636454  1.045581
在testing上的MSE,seed=407
       GT-1   GT-2(NN)       MDN
0  0.738199  13.223797  1.093239


## 3.3 Details: test set RMSE and MSE
1. 观察test set上每一个config
2. 表中的值均为**越小越好**

In [587]:
def get_diff(a, b):
    diff = a.odds - b.odds
    return diff

In [588]:
col_diff = ['GT1-target', 'MDN-target', 'GT1-MDN']
diff_arr = np.array([0.] * test_idx_len * 3).reshape(-1, 3)
diff_pd = pd.DataFrame(diff_arr, columns=col_diff)

In [ ]:
diff_pd[diff_pd["GT1-target"]<0]

In [589]:
diff_pd.iloc[:, 0] = get_diff(revenue_GT_1, revenue_target)
diff_pd.iloc[:, 1] = get_diff(revenue_MDN_pd, revenue_target)
diff_pd.iloc[:, 2] = get_diff(revenue_GT_1, revenue_MDN_pd)

In [590]:
diff_great = diff_pd[diff_pd['GT1-MDN'] <= 0]
diff_great.reset_index(inplace=True, drop=False)
diff_great.head()

,index,GT1-target,MDN-target,GT1-MDN
0,0,-0.998189,0.522893,-1.521082
1,2,-1.643688,-1.526670,-0.117019
2,3,-1.133977,-0.511939,-0.622038
3,4,-1.400101,-0.961067,-0.439034
4,5,-0.144227,0.851511,-0.995738


In [591]:
data_key.reset_index(inplace=True, drop=False)
diff_great_wkey = pd.merge(diff_great, data_key, on=['index'])
diff_great_wkey

,index,GT1-target,MDN-target,GT1-MDN,desc,bidincrement,bidfee,retail,flg_endprice
0,0,-0.998189,0.522893,-1.521082,Sony Ericsson S500i Unlocked Mysterious Green,0.15,0.75,499.99,0
1,2,-1.643688,-1.526670,-0.117019,iPod Touch Apple 8GB with Software Upgrade,0.15,0.75,299.99,0
2,3,-1.133977,-0.511939,-0.622038,Logitech Cordless Wave Keyboard and Mouse,0.15,0.75,89.99,0
3,4,-1.400101,-0.961067,-0.439034,SanDisk Cruzer Contour 4GB,0.15,0.75,59.99,0
4,5,-0.144227,0.851511,-0.995738,Mario Kart with Wheel (Nintendo Wii),0.15,0.75,49.99,0
...,...,...,...,...,...,...,...,...,...
159,246,0.937423,2.192028,-1.254605,Donkey Kong: Barrel Blast (Wii),0.15,0.75,44.98,0
160,248,0.944178,1.066811,-0.122633,Navigon 2200T 3.5-Inch Portable GPS Navigation,0.15,0.75,249.99,0
161,251,0.938333,1.088717,-0.150384,Wii Music (Wii),0.15,0.75,49.99,0
162,252,0.947896,1.216308,-0.268411,Fable II (Xbox 360),0.15,0.75,59.99,0
